In [1]:
import numpy as np
import glob
import os
from models.JetPointNet import PointNetSegmentation


def load_data_from_npz(npz_file):
    data = np.load(npz_file)
    feats = data['feats']  # Shape: (num_samples, 859, 6)
    frac_labels = data['frac_labels']  # Shape: (num_samples, 859)
    tot_labels = data['tot_labels']  # Shape: (num_samples, 859)
    tot_truth_e = data['tot_truth_e']  # Shape: (num_samples, 859) (This is the true total energy deposited by particles into this cell)
    return feats, frac_labels, tot_labels, tot_truth_e

# Setup
os.environ['CUDA_VISIBLE_DEVICES'] = ""  # Disable GPU
model_path = "saved_model/PointNetModel.keras"

TEST_DIR = '/data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test'

model = PointNetSegmentation(num_points=859, num_classes=1)
model.load_weights(model_path)

npz_files = glob.glob(os.path.join(TEST_DIR, '*.npz'))



2024-04-15 17:04:51.737612: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-15 17:04:51.737869: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: triumf-ml1
2024-04-15 17:04:51.737947: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: triumf-ml1
2024-04-15 17:04:51.738573: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 545.23.8
2024-04-15 17:04:51.738659: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  545.23.08  Release Build  (dvs-builder@U16-I3-A16-1-1)  Mon Nov  6 23:37:57 UTC 2023
GCC version:  gcc version 11.4.0 (Ubuntu 11.4.0-1u

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def plot_energies(predicted_energies, true_energies, coords, types, sample_index):
    fig = plt.figure(figsize=(14, 7))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    ax1.set_title(f'Sample {sample_index} - Predicted')
    ax2.set_title(f'Sample {sample_index} - Actual')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')

    vmax = max(np.max(predicted_energies), np.max(true_energies))
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=vmax))
    sm.set_array([])

    # Distinguish by types
    markers = {0: 'o', 1: '^', 2: 'x'}  # Example of different markers for different types

    for i, t in enumerate(types):
        if t == -1:
            continue  # Skip masked types
        color = 'r' if t == 2 else 'b' if t == 1 else 'g'  # Example colors
        marker = markers.get(t, 'o')
        ax1.scatter(coords[i, 0], coords[i, 1], coords[i, 2], color=color, s=40, marker=marker)
        ax2.scatter(coords[i, 0], coords[i, 1], coords[i, 2], color=color, s=40, marker=marker)

    cbar = fig.colorbar(sm, ax=[ax1, ax2], shrink=0.5, aspect=20)
    cbar.set_label('Energy (MeV)')

    plt.show()

for npz_file_idx, npz_file in enumerate(npz_files):
    if npz_file_idx > 1:
        break
    feats, _, labels, perfectly_measured_energy = load_data_from_npz(npz_file)
    segmentation_prediction, measured_energies = model.predict(feats)

    for sample_idx in range(min(10, len(feats))):
        types = feats[sample_idx][:, -1]
        valid_indices = types != -1
        energy_type_indices = types == 0  # Only consider type 0 for energy calculations

        coords = feats[sample_idx][valid_indices, :3]
        types_for_plotting = types[valid_indices]

        # Calculation of energies using the segmentation predictions and the measured energies
        predicted_energies_eval = segmentation_prediction[sample_idx] * measured_energies[sample_idx]
        predicted_energies_if_measurement_is_perfect = segmentation_prediction[sample_idx] * perfectly_measured_energy[sample_idx]

        # Calculating sums using only type 0 points
        sum_predicted_energy = np.sum(predicted_energies_eval[energy_type_indices])
        sum_predicted_energy_perfect = np.sum(predicted_energies_if_measurement_is_perfect[energy_type_indices])
        sum_true_energy = np.sum(labels[sample_idx][energy_type_indices])

        print(f"Sample {sample_idx}:")
        print(f"Sum of Predicted Energies: {sum_predicted_energy:.2f} MeV")
        print(f"Sum of Predicted Energies (Perfect Measurement): {sum_predicted_energy_perfect:.2f} MeV")
        print(f"Sum of True Energies: {sum_true_energy:.2f} MeV")
        print(f"Percentage Difference: {100 * np.abs(sum_predicted_energy - sum_true_energy) / sum_true_energy:.2f}% if sum_true_energy else 'NaN'%")



        print(labels[sample_idx][energy_type_indices])
        print(predicted_energies_eval[energy_type_indices])
        print(segmentation_prediction[sample_idx][energy_type_indices])
        #plot_energies(predicted_energies_eval[valid_indices], labels[sample_idx][valid_indices], coords, types_for_plotting, sample_idx)


13/13 [==============================] - 3s 260ms/step
Sample 0:
Sum of Predicted Energies: 1023.69 MeV
Sum of Predicted Energies (Perfect Measurement): 105151.95 MeV
Sum of True Energies: 7583.12 MeV
Percentage Difference: 86.50% if sum_true_energy else 'NaN'%
[[  0.85080975]
 [  2.590073  ]
 [  2.4958532 ]
 [ 27.742418  ]
 [  0.9556765 ]
 [  4.366705  ]
 [  1.8120601 ]
 [  0.26780692]
 [  1.4370061 ]
 [  1.543023  ]
 [  1.2283539 ]
 [  0.3217138 ]
 [  4.5173473 ]
 [  0.90165675]
 [  1.2284206 ]
 [  0.8551266 ]
 [  1.3362454 ]
 [  1.2756636 ]
 [  1.3861349 ]
 [  2.4523106 ]
 [  0.42675993]
 [  1.8153911 ]
 [  3.9342725 ]
 [  0.69292486]
 [  5.656532  ]
 [  1.3863254 ]
 [  1.9232513 ]
 [  0.6928794 ]
 [  3.245555  ]
 [  0.42680666]
 [  0.69298154]
 [  1.6545637 ]
 [  2.5086012 ]
 [  4.2194715 ]
 [  5.497366  ]
 [  0.80165714]
 [  6.3271    ]
 [  7.1985455 ]
 [  0.5876392 ]
 [  1.697592  ]
 [  0.90204054]
 [  4.630981  ]
 [  1.8619117 ]
 [  1.5437185 ]
 [  1.0104202 ]
 [  1.5441121 ]
 [

/tmp/ipykernel_2933755/360616125.py:66: RuntimeWarning: divide by zero encountered in scalar divide
  print(f"Percentage Difference: {100 * np.abs(sum_predicted_energy - sum_true_energy) / sum_true_energy:.2f}% if sum_true_energy else 'NaN'%")
